In [70]:
import os
import pandas as pd
import csv
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from konlpy.tag import Okt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from tqdm import tqdm
from gensim.models import Word2Vec
import time

# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자 변환 형식상 
    text = text.lower()
    # 특수 문자 제거
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    # 토큰화 및 불용어 제거 후 조사 명사 동사만 남기기
    tokens = okt.pos(text)
    tokens = [word for word, pos in tokens if pos in ['Josa', 'Noun', 'Verb']]
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

## 모델을 받아 변환을 하고 문서 용어 행렬을 반환하는 함수
def display_transform_dtm(cvect, corpus):
    print("Fitting and transforming the corpus...")
    dtm = cvect.fit_transform(tqdm(corpus, desc="Transforming corpus"))
    vocab = cvect.get_feature_names()
    print("Creating the DataFrame...")
    df_dtm = pd.DataFrame(dtm.toarray(), columns=vocab) ## 굳이 Dafatrame으로 보지않아도됨 시간 오래걸림 
    return df_dtm.style.background_gradient()


# 한국어 불용어 리스트 정의
stop_words = set([
    '의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', 
    '으로', '자', '에', '와', '한', '하다', '에서', '에게', '이다', '위해'
])

#예시 데이터 
corpus = ["코로나 거리두기와 코로나 상생지원금 문의입니다.",
          "지하철 운행시간과 지하철 요금 문의입니다.",
          "지하철 승강장 문의입니다.",
          "코로나 선별진료소 문의입니다.",
          "버스 운행시간 문의입니다.", 
          "버스 터미널 위치 안내입니다.",
          "코로나 거리두기 안내입니다.",
          "택시 승강장 문의입니다."
         ]

# 한국어 형태소 분석기 Okt 객체 생성
okt = Okt()

#측정시작 
start_time = time.time()

# CSV 파일 읽어서 저장 기사 Content 
file_path = 'result'
file_name = 'joongang_news(test).csv'
csv_file_path = os.path.join(file_path, file_name)

titles = []
with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        titles.append(row['Content'])
        
# 전처리된 제목 리스트 생성
processed_titles = [preprocess_text(title) for title in tqdm(titles, desc="Processing titles")]

# 생성된 제목 리스트로 Word2Vec 모델 학습
word2vec_model = Word2Vec(processed_titles, vector_size=100, window=5, min_count=1, workers=4)

# 전처리된 배열 출력 (일부만 출력)
#print(processed_titles[:10])

## cvect 단어 빈도수 기반 벡터 토크나이저 
cvect = CountVectorizer() 
## TF-IDF를 적용한 벡터 토크나이져 
tfidfvect = TfidfVectorizer()

# 파마메터 조정 확인용 
# ngram_range=(2, 3) : 토큰을 묶는 방식 파라메터 default 1:1
# min_df=2 : 해당 값보다 낮은 빈도수의 토큰(용어)은 무시한다. 
# max_df=0.7 : 너무 자주 등장하는 토큰은 무시한다. 
# stop_words=[] : 불용어 리스트, 해당 리스트에 있는 단어는 무시 
# analyzer='char' : char, char_wb 있음 

# 화면에 보여주기용 
#display(display_transform_dtm(tfidfvect, processed_titles))
#display_transform_dtm(cvect, titles)


# TF-IDF 벡터화 및 희소 행렬 생성
dtm = tfidfvect.fit_transform(processed_titles)

# 특징 이름(단어) 리스트
feature_names = tfidfvect.get_feature_names()

# 희소 행렬을 밀집 행렬로 변환
dense = dtm.todense()

# 밀집 행렬을 numpy 배열로 변환
denselist = dense.tolist()

# TF-IDF 결과 출력 (예제: 첫 10개 문서)
for i in range(10):
    print(f"문서 {i+1}의 TF-IDF 결과:")
    doc = denselist[i]
    phrase_scores = [pair for pair in zip(range(0, len(doc)), doc) if pair[1] > 0]
    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
    for phrase, score in sorted_phrase_scores:
        print(f"  {feature_names[phrase]}: {score}")
    print("\n")
    
    
# 입력 단어에 대한 유사 단어 찾기 및 TF-IDF 값 계산
def find_similar_words(input_word):
    similar_words = word2vec_model.wv.most_similar(input_word, topn=10)
    similar_words_tf_idf = {}
    for word, similarity in similar_words:
        if word in feature_names:
            index = feature_names.tolist().index(word)
            tf_idf_scores = [doc[index] for doc in denselist]
            similar_words_tf_idf[word] = tf_idf_scores
    return similar_words_tf_idf

# 예제: '코로나'와 유사한 단어 찾기
input_word = '코로나'
similar_words_tf_idf = find_similar_words(input_word)

# 유사 단어 및 TF-IDF 값 출력
for word, scores in similar_words_tf_idf.items():
    print(f"단어: {word}")
    print(f"TF-IDF 점수: {scores}")
    print("\n")
    
    
# 종료 시간 측정
end_time = time.time()
print(f"총 걸린 시간: {end_time - start_time} 초")


Processing titles:   1%|▊                                                        | 18/1234 [41:30<46:44:40, 138.39s/it]

Processing titles:   7%|████▌                                                        | 92/1234 [01:57<18:50,  1.01it/s]


Processing titles:  15%|████████▊                                                   | 181/1234 [03:27<12:09,  1.44it/s]

KeyboardInterrupt: 